# Data Exploration:
## Here:
- Handle missing values and outliers.
- Encode categorical variables.
- Normalize/standardize numerical features.
- Split the data into training and testing sets.


- Script: scripts/data_preprocessing.py   (This file contains necessary reusable codes and function that we use down below)

#### Settings to make scripts in scripts/ accessible.  
- this creates a .vscode in a root that includes the root path as well as __pycache__ in scripts directory

In [2]:
import sys
import os

# Add the project root directory to the system path
sys.path.append(os.path.abspath(os.path.join("..", "scripts")))

#### Necessary imports

In [3]:
import pandas as pd # type: ignore
import numpy as np # type: ignore
import data_preprocessing as dp  # This is a script file. 

#### Load data

In [4]:
df = dp.read_csv(path='../data/boston_housing_init.csv')
df.head(10)

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
5,10850000,7500,3,3,1,yes,no,yes,no,yes,2,yes,semi-furnished
6,10150000,8580,4,3,4,yes,no,no,no,yes,2,yes,semi-furnished
7,10150000,16200,5,3,2,yes,no,no,no,no,0,no,unfurnished
8,9870000,8100,4,1,2,yes,yes,yes,no,yes,2,yes,furnished
9,9800000,5750,3,2,4,yes,yes,no,no,yes,1,yes,unfurnished


#### Handle missing values and outliers

In [5]:
# dropping the missing values using the function coded in data_preprocessing.py

df = dp.handle_missing_values(df)
df.head(10)

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
5,10850000,7500,3,3,1,yes,no,yes,no,yes,2,yes,semi-furnished
6,10150000,8580,4,3,4,yes,no,no,no,yes,2,yes,semi-furnished
7,10150000,16200,5,3,2,yes,no,no,no,no,0,no,unfurnished
8,9870000,8100,4,1,2,yes,yes,yes,no,yes,2,yes,furnished
9,9800000,5750,3,2,4,yes,yes,no,no,yes,1,yes,unfurnished


In [6]:
# Checking and handling (if any) outliers using IQR statistics based on different column

# Outliers for the column: Price


outlier_price = dp.handling_outliers(df, 'price')
outlier_price

0     13300000
1     12250000
2     12250000
3     12215000
4     11410000
5     10850000
6     10150000
7     10150000
8      9870000
9      9800000
10     9800000
11     9681000
12     9310000
13     9240000
14     9240000
Name: price, dtype: int64

In [7]:
outlier_price.count()

15

In [8]:
# Outliers for the column: Area


outlier_area = dp.handling_outliers(df, 'area')
outlier_area

7      16200
10     13200
56     11440
64     11175
66     13200
69     12090
125    15600
129    11460
186    11410
191    10700
211    12900
403    12944
Name: area, dtype: int64

In [9]:
outlier_area.count()

12

#### Observation: with the IQR check in 2 different columns. Using the lower bounds and upper bounds techniques 15 and 12 data in 'price' and 'area' column respectively seems to be an outlier. Not touched since, this doesnot make an significant issue. 

#### Encode categorical variables

In [20]:
# Change categorical value of furnishingstatus into individual columns with the bool variables. 

furnishingstatus_decode =  dp.encode_categorical_vars(df, 'furnishingstatus')
furnishingstatus_decode

,furnished,semi-furnished,unfurnished
0,yes,no,no
1,yes,no,no
2,no,yes,no
3,yes,no,no
4,yes,no,no
...,...,...,...
540,no,no,yes
541,no,yes,no
542,no,no,yes
543,yes,no,no


In [11]:
# Merge the columns in a main dataframe and drop "Furnishing status"

df_merged = pd.concat([df, furnishingstatus_decode], axis=1).drop(columns='furnishingstatus')
df_merged.head(5)
 

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnished,semi-furnished,unfurnished
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,yes,no,no
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,yes,no,no
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,no,yes,no
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,yes,no,no
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,yes,no,no


#### Normalize/Standardize numerical features.

#### After encoding variables to each and every corelated column, let's change the boolean variables to 1 and 0. So that ML can efficiently perform. 

In [12]:
bool_columns = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnished', 'semi-furnished', 'unfurnished']
df_numeric = dp.encode_bools_to_binary(df=df_merged, bool_columns=bool_columns)

df_numeric

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnished,semi-furnished,unfurnished
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,1,0,0
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,1,0,0
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,0,1,0
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,1,0,0
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,1,0,1,0,0,2,0,0,0,1
541,1767150,2400,3,1,1,0,0,0,0,0,0,0,0,1,0
542,1750000,3620,2,1,1,1,0,0,0,0,0,0,0,0,1
543,1750000,2910,3,1,1,0,0,0,0,0,0,0,1,0,0


In [13]:
# Separate X and Y. X: Is everything expect the target, and Y is the target. i.e. Price
X = df_numeric.drop('price', axis=1) 
Y = df_numeric['price']

In [14]:
# Standardize data
X_scaled, scaler = dp.standardize_data(X)

#### Split the data into training and testing sets.

#### Splitted to 80%-20% (training - testing) data

In [15]:
X_train, X_test, y_train, y_test = dp.split_data(X_scaled, Y, 0.2)
print(X_train, X_test)

[[ 0.3917898   0.04727831  1.42181174 ...  1.70084013 -0.84488844
  -0.6964292 ]
 [ 0.94525725  0.04727831  1.42181174 ... -0.58794474  1.18358821
  -0.6964292 ]
 [-0.61552098 -1.30886273 -0.57018671 ...  1.70084013 -0.84488844
  -0.6964292 ]
 ...
 [-0.30004453  0.04727831  1.42181174 ...  1.70084013 -0.84488844
  -0.6964292 ]
 [-0.51220705 -1.30886273 -0.57018671 ... -0.58794474 -0.84488844
   1.43589615]
 [ 0.16117836  0.04727831  1.42181174 ... -0.58794474  1.18358821
  -0.6964292 ]] [[ 0.34566751  1.40341936  1.42181174 ... -0.58794474 -0.84488844
   1.43589615]
 [ 0.62240124  0.04727831  1.42181174 ...  1.70084013 -0.84488844
  -0.6964292 ]
 [-0.51220705 -1.30886273 -0.57018671 ... -0.58794474  1.18358821
  -0.6964292 ]
 ...
 [ 0.3917898   1.40341936  1.42181174 ... -0.58794474  1.18358821
  -0.6964292 ]
 [ 0.3917898   0.04727831  1.42181174 ... -0.58794474  1.18358821
  -0.6964292 ]
 [ 0.43791208  0.04727831  1.42181174 ...  1.70084013 -0.84488844
  -0.6964292 ]]


In [16]:
print(y_train, y_test)

46     7525000
93     6300000
335    3920000
412    3430000
471    3010000
        ...   
71     6755000
106    6160000
270    4340000
435    3290000
102    6195000
Name: price, Length: 436, dtype: int64 316    4060000
77     6650000
360    3710000
90     6440000
493    2800000
        ...   
15     9100000
357    3773000
39     7910000
54     7350000
155    5530000
Name: price, Length: 109, dtype: int64


#### Inorder to use testing and training data explicitely across the project, let us save the file in CSV format (.csv)

In [17]:
# Change np array to dataframe to save to CSV using dataframe

X_train_df = pd.DataFrame(X_train)
X_test_df = pd.DataFrame(X_test)

In [18]:
X_train_df.to_csv('../data/X_train.csv', index= False)
X_test_df.to_csv('../data/X_test.csv', index= False)

In [19]:
# Saving remaining np array without changing to pandas dataframes

np.savetxt('../data/y_train.csv',y_train, delimiter=',' )
np.savetxt('../data/y_test.csv',y_test, delimiter=',' )

#### 